In [18]:
from __future__ import unicode_literals
from __future__ import print_function
import pymongo
from pymongo import MongoClient
import json
import codecs
import re

In [22]:
client = MongoClient()
db = client['amazon']

In [23]:
with codecs.open('../crawl/headphone/headphone_2015-09-02T05-04-38.json', 'r', 'utf-8-sig') as fin:
    for i, line in enumerate(fin):
        item = json.loads(line.strip())
        item_type = item['item_type']
        
        # review_count
        if 'review_count' in item:
            item['review_count'] = int(item['review_count'].replace(',', ''))
            
        # helpful vote count
        if 'helpful_vote_count' in item:
            try:
                item['helpful_vote_count'] = [int(cnt.replace(',', '')) for cnt in re.findall(r'[0-9,]+', item['helpful_vote_count'])][0]
            except (TypeError, IndexError, Exception) as e:
                item['helpful_vote_count'] = 0
                continue
        
        collection = db['headphone_%s' % item_type]
        collection.insert(item)
        
        if not (i % 1000):
            print(i)

DuplicateKeyError: insertDocument :: caused by :: 11000 E11000 duplicate key error index: amazon.headphone_detail.$link_1  dup key: { : "http://www.amazon.com/Sony-MDR7506-Professional-Diaphragm-Headphone/dp/B000AJIF4E" }

# Create Index

In [21]:
db.headphone_detail.create_index("link", unique=True)
db.headphone_review.create_index([("item_link", pymongo.ASCENDING), ("helpful_vote_count", pymongo.DESCENDING)])

u'item_link_1_helpful_vote_count_-1'